# AER - Access Entitlement Review Reporter

從 SharePoint 讀取審核人員的 Excel 回應，產生缺漏報告與合併報告。

## Cell 1: 設定與認證

In [13]:
# === 設定與認證 (修改版：人肉登入模式) ===
import os
import logging
from datetime import datetime
from dotenv import load_dotenv
from msal import PublicClientApplication # <--- 改用 Public
import requests

load_dotenv()

# === Logging 設定 ===
os.makedirs("output", exist_ok=True)
log_file = f"output/aer_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

logger = logging.getLogger("aer")
logger.handlers.clear()  # 避免重複執行時累積 handler
logger.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')

ch = logging.StreamHandler()
ch.setFormatter(formatter)
logger.addHandler(ch)

fh = logging.FileHandler(log_file)
fh.setFormatter(formatter)
logger.addHandler(fh)

os.makedirs("output", exist_ok=True)
log_file = f"output/aer_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logger = logging.getLogger("aer")
logger.handlers.clear()
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')
ch = logging.StreamHandler()
ch.setFormatter(formatter)
logger.addHandler(ch)
fh = logging.FileHandler(log_file)
fh.setFormatter(formatter)
logger.addHandler(fh)

# === Azure AD 設定 ===
TENANT_ID = os.getenv("AZURE_TENANT_ID")
CLIENT_ID = os.getenv("AZURE_CLIENT_ID")
# CLIENT_SECRET 不需要了！因為是用你的人來登入
# CLIENT_SECRET = os.getenv("AZURE_CLIENT_SECRET") 

# === SharePoint 設定 ===
SHAREPOINT_HOST = os.getenv("SHAREPOINT_HOST", "davidshih.sharepoint.com")
SITE_NAME = os.getenv("SITE_NAME", "aer")
APP_NAME = "2025 Entitlement Review/Q4/AVIDXCHANGE-TEST"
BASE_PATH = APP_NAME

# === Email 設定 ===
# 寄件者必須是你登入的那個帳號 (Delegated 限制)
SENDER_EMAIL = os.getenv("SENDER_EMAIL") 

# === 取得 Access Token (改用 Device Code Flow) ===
# 這裡定義你需要的所有 Delegated 權限
SCOPES = ["Sites.Read.All", "Mail.Send", "User.Read.All"]

app = PublicClientApplication(
    CLIENT_ID,
    authority=f"https://login.microsoftonline.com/{TENANT_ID}"
)

# 1. 發起登入請求
flow = app.initiate_device_flow(scopes=SCOPES)

if "user_code" not in flow:
    raise ValueError("無法建立 Device Flow，請檢查 Azure App 是否開啟 'Allow public client flows'")

# 2. 顯示登入指示 (這就是你要手動做的地方)
print("\n" + "="*60)
print(f"👉 請打開瀏覽器，前往: {flow['verification_uri']}")
print(f"👉 輸入代碼 (User Code): {flow['user_code']}")
print("="*60 + "\n")
logger.info("等待使用者登入中...")

# 3. 等待你完成登入
token_result = app.acquire_token_by_device_flow(flow)

if "access_token" not in token_result:
    error_msg = token_result.get('error_description', 'Unknown error')
    logger.error(f"認證失敗: {error_msg}")
    raise Exception(f"Azure AD 認證失敗: {error_msg}")

headers = {"Authorization": f"Bearer {token_result['access_token']}"}
logger.info(f"🎉 登入成功！使用者: {token_result.get('id_token_claims', {}).get('name')}")
logger.info(f"處理 app: {APP_NAME}")
logger.info(f"Log 檔案: {log_file}")

2026-01-02 15:07:25,048 | INFO | 等待使用者登入中...



👉 請打開瀏覽器，前往: https://microsoft.com/devicelogin
👉 輸入代碼 (User Code): A8NGNYNNN



2026-01-02 15:07:40,589 | INFO | 🎉 登入成功！使用者: david shih
2026-01-02 15:07:40,590 | INFO | 處理 app: 2025 Entitlement Review/Q4/AVIDXCHANGE-TEST
2026-01-02 15:07:40,591 | INFO | Log 檔案: output/aer_20260102_150724.log


## Cell 2: SharePoint 函數

In [14]:
# === Cell 2: SharePoint 函數 (Level 3: 路徑修復版) ===

def get_site_id(site_name: str) -> str:
    """取得 SharePoint site ID"""
    url = f"https://graph.microsoft.com/v1.0/sites/{SHAREPOINT_HOST}:/sites/{site_name}"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()["id"]


def list_folders(site_id: str, path: str) -> list[dict]:
    """列出資料夾下的所有子資料夾（自動判斷是否為根目錄）"""
    
    # 修正邏輯：如果路徑是空的，代表要抓根目錄 (Root)
    if not path or path.strip() == "":
        url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root/children"
    else:
        # 如果有路徑，才用冒號語法
        # 移除前後可能多餘的斜線
        clean_path = path.strip("/")
        url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{clean_path}:/children"

    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    
    return [
        {"name": item["name"], "webUrl": item.get("webUrl", "")}
        for item in resp.json().get("value", [])
        if item.get("folder")
    ]


def list_excel_files(site_id: str, folder_path: str) -> list[dict]:
    """列出資料夾下的 Excel 檔案"""
    # 同樣修正路徑邏輯
    clean_path = folder_path.strip("/")
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{clean_path}:/children"
    
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    
    files = []
    for item in resp.json().get("value", []):
        if item["name"].endswith(".xlsx"):
            files.append({
                "id": item["id"],
                "name": item["name"],
                "lastModifiedDateTime": item.get("lastModifiedDateTime"),
                "webUrl": item.get("webUrl")
            })
    return sorted(files, key=lambda f: f.get("lastModifiedDateTime", ""), reverse=True)


def download_file(site_id: str, file_path: str) -> bytes:
    """下載檔案內容"""
    clean_path = file_path.strip("/")
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{clean_path}:/content"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.content


def get_file_audit_log(site_id: str, file_id: str) -> str:
    """取得檔案版本歷史 (Audit Log)"""
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/items/{file_id}/versions"
    resp = requests.get(url, headers=headers)
    
    if resp.status_code != 200:
        return "無法取得版本紀錄"

    versions = resp.json().get("value", [])
    logs = []
    for v in versions:
        mod_time = v.get("lastModifiedDateTime", "")[:19].replace("T", " ")
        user_info = v.get("lastModifiedBy", {}).get("user", {})
        app_info = v.get("lastModifiedBy", {}).get("application", {})
        actor = user_info.get("displayName") or app_info.get("displayName") or "Unknown System"
        ver_num = v.get("id", "")
        logs.append(f"{mod_time} - {actor} (v{ver_num})")
        
    return "\n".join(logs)

# 重新測試連線
try:
    site_id = get_site_id(SITE_NAME)
    logger.info(f"SharePoint 連線成功，Site ID: {site_id[:20]}...")
    logger.info("函數庫已更新 (路徑修正版)")
except Exception as e:
    logger.error(f"SharePoint 連線失敗: {e}")
    raise

2026-01-02 15:07:47,033 | INFO | SharePoint 連線成功，Site ID: davidshih.sharepoint...
2026-01-02 15:07:47,034 | INFO | 函數庫已更新 (路徑修正版)


In [ ]:
# === Cell 2.5: 互動式資料夾選擇器 (Folder Browser 修正版) ===
import ipywidgets as widgets
from IPython.display import display, clear_output

# 🔥 關鍵修正：起始路徑設為空字串，代表直接從 drive/root 開始
current_path = "" 
selected_target_path = None

def create_folder_browser():
    if 'list_folders' not in globals() or 'site_id' not in globals():
        print("⚠️ 請先執行 Cell 2 以載入 SharePoint 函式！")
        return

    # --- UI 元件 ---
    path_label = widgets.HTML(value=f"<b>目前路徑:</b> Root (Shared Documents)")
    
    folder_dropdown = widgets.Dropdown(
        options=[],
        description='子資料夾:',
        disabled=False,
        layout=widgets.Layout(width='50%')
    )
    
    btn_enter = widgets.Button(description='📂 進入', button_style='info', icon='folder-open')
    btn_back = widgets.Button(description='⬅️ 返回', button_style='', icon='arrow-left')
    btn_select = widgets.Button(description='✅ 設定為目標資料夾', button_style='success', icon='check')
    
    msg_output = widgets.Output()

    # --- 功能邏輯 ---
    def load_folders(path):
        try:
            items = list_folders(site_id, path)
            options = [item['name'] for item in items if item['name'] != "Forms"]
            folder_dropdown.options = sorted(options)
            
            if not options:
                folder_dropdown.disabled = True
                folder_dropdown.placeholder = "無子資料夾"
            else:
                folder_dropdown.disabled = False
                folder_dropdown.value = options[0] if options else None
                
        except Exception as e:
            with msg_output:
                print(f"❌ 讀取失敗: {e}")

    def on_enter_click(b):
        global current_path
        if folder_dropdown.value:
            # 修正路徑串接邏輯
            if current_path:
                current_path = f"{current_path}/{folder_dropdown.value}"
            else:
                current_path = folder_dropdown.value
                
            path_label.value = f"<b>目前路徑:</b> {current_path}"
            load_folders(current_path)

    def on_back_click(b):
        global current_path
        if "/" in current_path:
            current_path = current_path.rsplit("/", 1)[0]
        else:
            current_path = "" # 回到 Root
            
        path_label.value = f"<b>目前路徑:</b> {current_path or 'Root (Shared Documents)'}"
        load_folders(current_path)

    def on_select_click(b):
        global BASE_PATH
        BASE_PATH = current_path
        with msg_output:
            clear_output()
            print(f"🎯 已鎖定目標路徑: {BASE_PATH}")
            print("👉 請繼續執行 Cell 3 和 Cell 4。")

    btn_enter.on_click(on_enter_click)
    btn_back.on_click(on_back_click)
    btn_select.on_click(on_select_click)

    load_folders(current_path)

    ui = widgets.VBox([
        widgets.HTML("<h3>📂 SharePoint 資料夾瀏覽器</h3>"),
        path_label,
        widgets.HBox([btn_back, folder_dropdown, btn_enter]),
        widgets.HTML("<br>"),
        btn_select,
        msg_output
    ])
    display(ui)

create_folder_browser()

## Cell 3: Excel 讀取

In [16]:
# === Excel 讀取（處理 AutoFilter visible rows）===
from openpyxl import load_workbook
from io import BytesIO

# 欄位名稱設定
COL_REVIEWER = "Reviewer"
COL_RESPONSE = "Reviewer's Response"
COL_DETAILS = "Details of Access change"


def read_visible_rows(excel_bytes: bytes, reviewer_name: str, file_name: str, folder_url: str) -> list[dict]:
    """
    讀取 Excel 的 visible rows（考慮 AutoFilter）
    
    Args:
        excel_bytes: Excel 檔案的 bytes
        reviewer_name: 審核人名稱（用於 fallback 過濾）
        file_name: 檔案名稱（追蹤用）
        folder_url: SharePoint 資料夾 URL（可點擊）
    
    Returns:
        list of ReviewerResponse dict
    """
    wb = load_workbook(BytesIO(excel_bytes))
    ws = wb.active
    
    # 找出欄位索引（假設第一行是標題）
    header_row = [cell.value for cell in ws[1]]
    col_map = {str(name).strip(): idx for idx, name in enumerate(header_row) if name}
    
    # 檢查必要欄位
    reviewer_col = col_map.get(COL_REVIEWER)
    response_col = col_map.get(COL_RESPONSE)
    details_col = col_map.get(COL_DETAILS)
    
    if reviewer_col is None or response_col is None:
        raise ValueError(f"找不到必要欄位。現有欄位: {list(col_map.keys())}")
    
    # 檢查是否有 hidden rows（表示有 AutoFilter）
    has_filter = any(
        ws.row_dimensions[i].hidden 
        for i in range(2, ws.max_row + 1) 
        if i in ws.row_dimensions
    )
    
    results = []
    for row_idx in range(2, ws.max_row + 1):
        # 如果有 filter，跳過被隱藏的行
        if has_filter and ws.row_dimensions.get(row_idx) and ws.row_dimensions[row_idx].hidden:
            continue
        
        row = [cell.value for cell in ws[row_idx]]
        
        # 取得欄位值
        reviewer = row[reviewer_col] if reviewer_col < len(row) else None
        response = row[response_col] if response_col < len(row) else None
        details = row[details_col] if details_col is not None and details_col < len(row) else None
        
        # 如果沒有 filter，用欄位值過濾
        if not has_filter:
            if reviewer is None or str(reviewer).strip().lower() != reviewer_name.lower():
                continue
        
        # 判斷是否缺漏
        is_missing = response is None or str(response).strip() == ""
        
        results.append({
            "reviewer": reviewer_name,
            "response": response,
            "details": details,
            "is_missing": is_missing,
            "row_number": row_idx,
            "file_name": file_name,
            "folder_url": folder_url
        })
    
    wb.close()
    return results


logger.info("Excel 讀取函數已載入")

2026-01-02 15:08:10,006 | INFO | Excel 讀取函數已載入


## Cell 4: 資料收集

In [18]:
# === Cell 4: 資料收集 (智慧偵測版：支援 App 層級與 User 層級) ===
import pandas as pd

# 確保 BASE_PATH 是正確的相對路徑
logger.info(f"目前設定路徑: {BASE_PATH}")
initial_folders = list_folders(site_id, BASE_PATH)

# === 智慧判斷邏輯 ===
# 判斷我們現在是在 "Q4" (看到一堆 App) 還是在 "App" (看到一堆 User)
# 方法：檢查第一個資料夾，如果裡面包含 "審核 Excel"，代表我們選對了。
# 如果裡面包含的是 "資料夾"，代表我們選太高層了。

target_app_folders = [] # 格式: (App名稱, 完整路徑)

# 先偷看第一個資料夾的內容
if initial_folders:
    first_folder_name = initial_folders[0]["name"]
    first_folder_path = f"{BASE_PATH}/{first_folder_name}"
    
    # 看看裡面有沒有 Excel
    files_inside = list_excel_files(site_id, first_folder_path)
    is_user_folder = any(first_folder_name.lower() in f["name"].lower() for f in files_inside)
    
    if is_user_folder:
        # 狀況 A: 我們選到了 App 層級 (裡面直接是 User) -> 舊邏輯
        logger.info("✅ 偵測到直接路徑 (Single App Mode)")
        target_app_folders.append((APP_NAME, BASE_PATH))
    else:
        # 狀況 B: 我們選到了 Q4 層級 (裡面是 App 資料夾) -> 批次邏輯
        logger.info("🔄 偵測到上層路徑 (Batch App Mode) - 將掃描所有子 App")
        for f in initial_folders:
            # 把 Q4 底下的每個 App 都加入排程
            app_sub_path = f"{BASE_PATH}/{f['name']}"
            target_app_folders.append((f['name'], app_sub_path))
else:
    logger.warning("❌ 目標路徑下是空的！")

# === 開始跑迴圈 ===
all_responses = []
errors = []

for current_app_name, current_path in target_app_folders:
    logger.info(f"🚀 開始處理 App: {current_app_name} (路徑: {current_path})")
    
    # 抓取該 App 下的所有審核人
    try:
        reviewers = list_folders(site_id, current_path)
    except Exception as e:
        logger.error(f"  無法讀取 App 資料夾: {e}")
        continue
        
    for folder in reviewers:
        reviewer_name = folder["name"]
        folder_url = folder["webUrl"]
        folder_path = f"{current_path}/{reviewer_name}"
        
        try:
            # 列出 Excel 檔案
            excel_files = list_excel_files(site_id, folder_path)
            
            # 找到包含審核人名字的 Excel 檔案
            target_files = [f for f in excel_files if reviewer_name.lower() in f["name"].lower()]
            
            if not target_files:
                # 靜默跳過系統資料夾或不相關的
                if reviewer_name not in ["Forms", "_private"]:
                    logger.warning(f"  [跳過] {reviewer_name}: 找不到對應 Excel")
                continue
            
            # 讀取最新的檔案
            target_file = target_files[0]
            file_name = target_file["name"]
            file_id = target_file["id"]
            file_download_path = f"{folder_path}/{file_name}"
            
            logger.info(f"  📖 讀取: {current_app_name} -> {reviewer_name}")
            
            # 抓取 Audit Log
            audit_log_str = get_file_audit_log(site_id, file_id)
            
            # 下載並解析
            excel_bytes = download_file(site_id, file_download_path)
            rows = read_visible_rows(excel_bytes, reviewer_name, file_name, folder_url)
            
            # 補上 App Name 欄位 (這樣合併報告才知道是哪個 App)
            for row in rows:
                row["App_Name"] = current_app_name
                row["Audit_History"] = audit_log_str
                row["Last_Modified"] = target_file["lastModifiedDateTime"]
            
            all_responses.extend(rows)
            
        except Exception as e:
            logger.error(f"  ❌ 失敗 {reviewer_name}: {e}")
            errors.append({"reviewer": reviewer_name, "error": str(e), "folder_url": folder_url})

# 轉成 DataFrame
df = pd.DataFrame(all_responses)
logger.info(f"\n=== 收集完成 ===")
logger.info(f"總計: {len(all_responses)} 筆記錄")
logger.info(f"缺漏: {df['is_missing'].sum() if len(df) > 0 else 0} 筆")

2026-01-02 15:33:37,418 | INFO | 目前設定路徑: 2025 Entitlement Review/Q4
2026-01-02 15:33:38,084 | INFO | 🔄 偵測到上層路徑 (Batch App Mode) - 將掃描所有子 App
2026-01-02 15:33:38,085 | INFO | 🚀 開始處理 App: AVIDXCHANGE-TEST (路徑: 2025 Entitlement Review/Q4/AVIDXCHANGE-TEST)
2026-01-02 15:33:38,723 | INFO |   📖 讀取: AVIDXCHANGE-TEST -> Bob O'Brien
2026-01-02 15:33:40,171 | INFO |   📖 讀取: AVIDXCHANGE-TEST -> Jane Smith
2026-01-02 15:33:41,333 | INFO |   📖 讀取: AVIDXCHANGE-TEST -> Joe Petti
2026-01-02 15:33:42,236 | INFO | 🚀 開始處理 App: Visa (路徑: 2025 Entitlement Review/Q4/Visa)
2026-01-02 15:33:43,280 | INFO |   📖 讀取: Visa -> Alice Wonderland
2026-01-02 15:33:44,645 | INFO |   📖 讀取: Visa -> Charlie Brown
2026-01-02 15:33:45,568 | INFO | 
=== 收集完成 ===
2026-01-02 15:33:45,569 | INFO | 總計: 22 筆記錄
2026-01-02 15:33:45,569 | INFO | 缺漏: 5 筆


## Cell 5: 互動式狀態表格

In [23]:
# === Cell 5: 互動式狀態表格 (Q4 層級顯示版) ===
from IPython.display import display, HTML
import pandas as pd

def create_status_table(df: pd.DataFrame) -> pd.DataFrame:
    """建立互動式狀態表格 (含上層目錄顯示)"""
    
    if len(df) == 0:
        display(HTML("<p style='color: red;'>沒有資料可顯示</p>"))
        return pd.DataFrame()
    
    # --- 1. 抓取層級名稱 (例如 "Q4") ---
    # 從 BASE_PATH 抓最後一層資料夾名稱
    try:
        # 移除可能存在的尾端斜線，然後抓最後一段
        parent_folder = BASE_PATH.strip("/").split("/")[-1]
    except:
        parent_folder = "Root"
        
    # 防呆
    if "App_Name" not in df.columns:
        df["App_Name"] = APP_NAME

    # --- 2. 組合顯示名稱 ---
    # 建立一個暫存欄位用來顯示： "Q4 / AppName"
    # 如果 App Name 本身已經包含路徑，我們取最後一段再組合
    df["Display_App_Name"] = df["App_Name"].apply(
        lambda x: f"{parent_folder} / {x.split('/')[-1] if '/' in x else x}"
    )

    # --- 3. 統計與分組 ---
    # 這裡我們用 Display_App_Name 來分組
    summary = df.groupby(["Display_App_Name", "reviewer", "folder_url"]).agg(
        total=("reviewer", "count"),
        missing=("is_missing", "sum")
    ).reset_index()
    
    summary["status"] = summary.apply(
        lambda r: "🔴 未完成" if r["missing"] > 0 else "🟢 完成", axis=1
    )
    summary["completion"] = ((summary["total"] - summary["missing"]) / summary["total"] * 100).round(1)
    
    # --- 4. 排序邏輯 ---
    # 1. 先把有缺漏(未完成)的 App 排前面
    # 2. 同樣狀態下，照 App 名稱排
    # 3. 同一個 App 內，照審核人名字排
    summary_sorted = summary.sort_values(
        ["missing", "Display_App_Name", "reviewer"], 
        ascending=[False, True, True]
    )
    
    # --- 5. 建立 HTML ---
    def make_row(r):
        bg_color = "#ffcccc" if r["missing"] > 0 else "#ccffcc"
        return f"""
        <tr style="background-color: {bg_color};">
            <td style="font-weight:bold;">{r['Display_App_Name']}</td>
            <td><a href="{r['folder_url']}" target="_blank">{r['reviewer']}</a></td>
            <td>{r['status']}</td>
            <td>{r['completion']}%</td>
            <td>{int(r['missing'])} / {int(r['total'])}</td>
        </tr>
        """
    
    html = f"""
    <style>
        .status-table {{ border-collapse: collapse; width: 100%; font-family: sans-serif; }}
        .status-table th, .status-table td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
        .status-table th {{ background-color: #4CAF50; color: white; }}
        .status-table a {{ color: #0066cc; text-decoration: none; font-weight: bold; }}
        .status-table a:hover {{ text-decoration: underline; }}
    </style>
    <h3>📊 審核狀態總覽</h3>
    <table class="status-table">
        <tr>
            <th>App 名稱 (層級)</th>
            <th>審核人</th>
            <th>狀態</th>
            <th>完成率</th>
            <th>缺漏/總數</th>
        </tr>
        {''.join(summary_sorted.apply(make_row, axis=1))}
    </table>
    <p>
        <b>總計:</b> {len(summary)} 位審核人, 
        <b>{int(summary['missing'].sum())}</b> 筆缺漏
    </p>
    """
    
    display(HTML(html))
    return summary

# 執行
summary_df = create_status_table(df)

App 名稱 (層級),審核人,狀態,完成率,缺漏/總數
Q4 / AVIDXCHANGE-TEST,Joe Petti,🔴 未完成,50.0%,2 / 4
Q4 / Visa,Charlie Brown,🔴 未完成,60.0%,2 / 5
Q4 / AVIDXCHANGE-TEST,Jane Smith,🔴 未完成,80.0%,1 / 5
Q4 / AVIDXCHANGE-TEST,Bob O'Brien,🟢 完成,100.0%,0 / 3
Q4 / Visa,Alice Wonderland,🟢 完成,100.0%,0 / 5


## Cell 6: 發送提醒郵件

In [ ]:
# === Cell 6: 互動式催稿控制台 ===
# 如果看不到 UI，請執行以下步驟:
# 1. Kernel -> Restart Kernel
# 2. 重新執行所有 Cell
# 3. 如果還是不行，在終端機執行: jupyter labextension list

import ipywidgets as widgets
from IPython.display import display, HTML
from urllib.parse import quote

# === 診斷: 檢查 ipywidgets 是否正常 ===
print("ipywidgets 版本:", widgets.__version__)
test_btn = widgets.Button(description="測試按鈕 - 如果看到這個按鈕表示正常")
display(test_btn)

# === 防呆檢查 ===
if 'summary_df' not in dir() or len(summary_df) == 0:
    display(HTML("<p style='color:red'>⚠️ 請先執行 Cell 4 和 Cell 5</p>"))
else:
    # === 設定區 ===
    try:
        default_domain = SENDER_EMAIL.split("@")[1] if SENDER_EMAIL else "company.com"
    except:
        default_domain = "company.com"

    def resolve_reviewer_email(name: str, fallback_domain: str) -> tuple:
        """取得審核人 Email (Graph API -> Fallback)"""
        try:
            encoded_name = quote(name)
            url = f"https://graph.microsoft.com/v1.0/users?$filter=displayName eq '{encoded_name}'"
            resp = requests.get(url, headers=headers)
            if resp.status_code == 200:
                users = resp.json().get("value", [])
                if users:
                    email = users[0].get("mail") or users[0].get("userPrincipalName")
                    if email:
                        return email, "AzureAD"
        except:
            pass
        
        # Fallback
        try:
            parts = name.strip().split()
            if len(parts) >= 2:
                fmt_name = f"{parts[0]}.{parts[-1]}".lower()
            else:
                fmt_name = parts[0].lower()
            return f"{fmt_name}@{fallback_domain}", "Fallback"
        except:
            return "", "Error"

    def send_api_mail(to_email: str, subject: str, body_content: str) -> bool:
        """呼叫 Graph API 寄信"""
        if not SENDER_EMAIL:
            return False
        url = f"https://graph.microsoft.com/v1.0/users/{SENDER_EMAIL}/sendMail"
        data = {
            "message": {
                "subject": subject,
                "body": {"contentType": "HTML", "content": body_content},
                "toRecipients": [{"emailAddress": {"address": to_email}}]
            }
        }
        resp = requests.post(url, headers={**headers, "Content-Type": "application/json"}, json=data)
        return resp.status_code == 202

    # === UI 元件 ===
    domain_input = widgets.Text(
        value=default_domain, 
        description='Fallback網域:', 
        placeholder='e.g. company.com',
        style={'description_width': '100px'}
    )
    
    subject_input = widgets.Text(
        value="[Action Required] Access Review Reminder", 
        description='主旨:', 
        layout=widgets.Layout(width='80%'),
        style={'description_width': '50px'}
    )
    
    default_template = """<p>Hi {name},</p>
<p>這是一個溫馨提醒，您目前還有 <b>{missing}</b> 筆權限審核尚未完成。</p>
<p>請點擊此連結進行審核：<a href="{link}">開啟資料夾</a></p>
<p>謝謝</p>"""
    
    body_input = widgets.Textarea(
        value=default_template, 
        description='內容模板:', 
        layout=widgets.Layout(width='90%', height='120px'),
        style={'description_width': '80px'}
    )
    
    log_output = widgets.Output(layout={'border': '1px solid #ddd', 'min_height': '80px'})
    
    # === 建立審核人列表 ===
    targets = summary_df[summary_df['missing'] > 0].copy()
    
    if len(targets) == 0:
        display(HTML("<p style='color:green; font-size:18px'>🎉 所有審核人都已完成，無需寄信！</p>"))
    else:
        checkboxes = {}
        rows_widgets = []
        
        for _, row in targets.iterrows():
            r_name = row['reviewer']
            r_miss = int(row['missing'])
            r_link = row['folder_url']
            email, source = resolve_reviewer_email(r_name, default_domain)
            
            cb = widgets.Checkbox(value=False, description='', layout=widgets.Layout(width='30px'))
            checkboxes[r_name] = {'cb': cb, 'email': email, 'missing': r_miss, 'link': r_link}
            
            color = "blue" if source == "AzureAD" else "orange"
            label = widgets.HTML(
                f"<span style='width:150px;display:inline-block'><b>{r_name}</b></span> "
                f"<span style='width:60px;display:inline-block'>({r_miss}筆)</span> "
                f"<span style='color:{color}'>{email}</span> <small>({source})</small>"
            )
            
            rows_widgets.append(widgets.HBox([cb, label]))
        
        # 全選
        select_all = widgets.Checkbox(value=False, description='全選')
        def on_select_all(change):
            for data in checkboxes.values():
                data['cb'].value = change['new']
        select_all.observe(on_select_all, names='value')
        
        # 發送按鈕
        send_btn = widgets.Button(description='發送選中的提醒', button_style='warning', icon='envelope')
        
        def on_send(b):
            with log_output:
                log_output.clear_output()
                sent = 0
                for name, data in checkboxes.items():
                    if data['cb'].value and data['email']:
                        print(f"發送給 {name} ({data['email']})...")
                        body = body_input.value.format(name=name, missing=data['missing'], link=data['link'])
                        if send_api_mail(data['email'], subject_input.value, body):
                            print(f"  ✅ 成功")
                            sent += 1
                        else:
                            print(f"  ❌ 失敗")
                print(f"\n完成！共發送 {sent} 封郵件")
        
        send_btn.on_click(on_send)
        
        # === 組合 UI ===
        ui = widgets.VBox([
            widgets.HTML("<h3>📧 審核催稿中心</h3>"),
            widgets.HBox([domain_input]),
            subject_input,
            body_input,
            widgets.HTML("<hr><b>選擇要發送提醒的審核人:</b>"),
            select_all,
            widgets.VBox(rows_widgets),
            widgets.HTML("<br>"),
            send_btn,
            widgets.HTML("<b>執行紀錄:</b>"),
            log_output
        ])
        
        display(ui)

## Cell 7: 產生報告

In [24]:
# === Cell 7: 產生報告 (含智慧型欄寬調整) ===
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
import pandas as pd # 確保有引入

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# 處理檔名非法字元
safe_app_name = APP_NAME.replace("/", "_").replace("\\", "_")

def format_excel_layout(file_path: str):
    """
    [美化函數] 智慧型欄寬調整：
    1. 長文欄位 (Audit, Details) -> 固定寬度 + 自動換行
    2. 短文欄位 (Reviewer, Date) -> 自動偵測內容長度 (AutoFit)
    """
    try:
        wb = load_workbook(file_path)
        ws = wb.active
        
        # 定義需要「固定寬度 + 換行」的長欄位名稱
        long_text_cols = ["Audit_History", "Details of Access change", "details", "Details", "response"]
        
        # 遍歷每一欄 (Column)
        for column_cells in ws.columns:
            # 取得該欄的第一格 (標題)
            header_cell = column_cells[0]
            header_val = str(header_cell.value).strip() if header_cell.value else ""
            col_letter = get_column_letter(header_cell.column)
            
            # === 策略 A: 長文欄位 (固定寬度 + 換行) ===
            if header_val in long_text_cols:
                ws.column_dimensions[col_letter].width = 50 # 固定寬度
                # 設定整欄自動換行 + 靠上對齊
                for cell in column_cells:
                    cell.alignment = Alignment(wrap_text=True, vertical='top')
            
            # === 策略 B: 一般欄位 (AutoFit) ===
            else:
                max_length = 0
                for cell in column_cells:
                    try:
                        if cell.value:
                            # 只有標題列加一點 buffer，避免篩選按鈕遮住字
                            cell_len = len(str(cell.value))
                            if cell_len > max_length:
                                max_length = cell_len
                    except:
                        pass
                
                # 設定寬度 (字數 * 1.2 倍左右比較剛好，並加一點緩衝)
                adjusted_width = (max_length + 2) * 1.1
                # 設個上限，避免意外過寬
                if adjusted_width > 60: adjusted_width = 60
                
                ws.column_dimensions[col_letter].width = adjusted_width
                
                # 雖然不換行，但還是靠上對齊比較整齊
                for cell in column_cells:
                    current_align = cell.alignment
                    # 保留原本的設定，只加 Vertical Top
                    cell.alignment = Alignment(vertical='top', horizontal=current_align.horizontal)

        wb.save(file_path)
        logger.info(f"✨ 已美化 Excel (AutoFit + Wrap Text): {file_path}")
        
    except Exception as e:
        logger.warning(f"Excel 格式美化失敗 (但不影響檔案內容): {e}")


# === 主要邏輯 ===
if len(df) > 0:
    # 定義輸出的欄位順序 (加上 Display_App_Name 比較好辨識)
    export_cols = [
        "App_Name", "reviewer", "response", "details", "is_missing", 
        "Audit_History", "Last_Modified", 
        "row_number", "file_name", "folder_url"
    ]
    
    # 1. 缺漏報告
    missing_df = df[df["is_missing"]].copy()
    missing_file_xlsx = f"output/missing_responses_{safe_app_name}_{timestamp}.xlsx"
    
    # 只輸出存在的欄位
    cols_to_save = [c for c in export_cols if c in missing_df.columns]
    missing_df[cols_to_save].to_excel(missing_file_xlsx, index=False)
    format_excel_layout(missing_file_xlsx) # <--- 呼叫智慧美化
    
    # 2. 合併報告
    consolidated_file_xlsx = f"output/consolidated_report_{safe_app_name}_{timestamp}.xlsx"
    
    cols_to_save = [c for c in export_cols if c in df.columns]
    df[cols_to_save].to_excel(consolidated_file_xlsx, index=False)
    format_excel_layout(consolidated_file_xlsx) # <--- 呼叫智慧美化
    
    display(HTML(f"""
    <h3>報告已產生 (含 AutoFit 欄寬)</h3>
    <ul>
        <li>缺漏報告: <code>{missing_file_xlsx}</code></li>
        <li>合併報告: <code>{consolidated_file_xlsx}</code></li>
    </ul>
    """))

else:
    logger.warning("沒有資料可產生報告")
    display(HTML("<p style='color: orange;'>沒有資料可產生報告</p>"))

2026-01-02 15:41:07,256 | INFO | ✨ 已美化 Excel (AutoFit + Wrap Text): output/missing_responses_2025 Entitlement Review_Q4_AVIDXCHANGE-TEST_20260102_154107.xlsx
2026-01-02 15:41:07,276 | INFO | ✨ 已美化 Excel (AutoFit + Wrap Text): output/consolidated_report_2025 Entitlement Review_Q4_AVIDXCHANGE-TEST_20260102_154107.xlsx


## Cell 8: 錯誤報告

In [ ]:
# === 錯誤報告 ===

if errors:
    error_df = pd.DataFrame(errors)
    error_file = f"output/errors_{APP_NAME}_{timestamp}.csv"
    error_df.to_csv(error_file, index=False)
    
    logger.warning(f"有 {len(errors)} 個審核人處理失敗")
    
    # 顯示錯誤表格
    def make_error_row(r):
        return f"""
        <tr style="background-color: #fff3cd;">
            <td><a href="{r['folder_url']}" target="_blank">{r['reviewer']}</a></td>
            <td>{r['error']}</td>
        </tr>
        """
    
    html = f"""
    <h3>⚠️ 處理錯誤</h3>
    <table class="status-table">
        <tr>
            <th>審核人</th>
            <th>錯誤訊息</th>
        </tr>
        {''.join(error_df.apply(make_error_row, axis=1))}
    </table>
    <p>錯誤報告已儲存: <code>{error_file}</code></p>
    """
    display(HTML(html))
else:
    display(HTML("<p style='color: green;'>✅ 所有審核人都處理成功！</p>"))